Work in progress!

# The Acrobot

*We recommend you look at the [Introduction to Crocoddyl](introduction_to_crocoddyl.ipynb) example before this one.*

In the example, we model the acrobot control problem using Crocoddyl. An acrobot is a two joint planar robot with only one actuator. It is a canonnical example of an underactuated system and so presents an interesting control problem.

We demonstrate how to:

1. Load a model from an urdf.
1. Define an actuation mapping for the system.
1. Construct and solve the control problem.

## Loading the model
A standalone double pendulum robot urdf is provided in the [example-robot-data](https://github.com/Gepetto/example-robot-data) repository, this comes bundled with Crocoddyl. Let's load the model and inspect its properties.

In [ ]:
import os
import pathlib

import numpy as np
import pinocchio

# Get the path to the urdf
from example_robot_data.path import EXAMPLE_ROBOT_DATA_MODEL_DIR

import crocoddyl

urdf_model_path = pathlib.Path(
    "double_pendulum_description", "urdf", "double_pendulum_simple.urdf"
)
urdf_model_path = os.path.join(EXAMPLE_ROBOT_DATA_MODEL_DIR, urdf_model_path)

# Now load the model (using pinocchio)
robot = pinocchio.robot_wrapper.RobotWrapper.BuildFromURDF(str(urdf_model_path))

# The model loaded from urdf (via pinicchio)
print(robot.model)

# Create a multibody state from the pinocchio model.
state = crocoddyl.StateMultibody(robot.model)

`pinocchio` comes with some handy wrappers that load a robot along with vizual and collision models. These are all defined via that urdf. `robot.model` is the model is a model of the DAEs (Differential Algebraic Equations).

You will notice that the there are two joint configurations `nq` and velocities `nv`.

## Actuation Mapping
In order to create an underactuated double pendulum, the acrobot, we will create mapping between control inputs and joint torques. This is done by inheriting from `ActuationModelAbstract`. See also `ActuationModelFloatingBase` and `ActuationModelFull` for other options.

In [ ]:
# Define the control signal to actuated joint mapping
class AcrobotActuationModel(crocoddyl.ActuationModelAbstract):
    def __init__(self, state):
        nu = 1  # Control dimension
        crocoddyl.ActuationModelAbstract.__init__(self, state, nu=nu)

    def calc(self, data, x, u):
        assert len(data.tau) == 2
        # Map the control dimensions to the joint torque
        data.tau[0] = 0
        data.tau[1] = u

    def calcDiff(self, data, x, u):
        # Specify the actuation jacobian
        data.dtau_du[0] = 0
        data.dtau_du[1] = 1


# Also see ActuationModelFloatingBase and ActuationModelFull
actuationModel = AcrobotActuationModel(state)

## Constructing the Problem

Before we solve the control problem, we need to construct the cost models and action models.

In [ ]:
dt = 1e-3  # Time step
T = 1000  # Number of knots

# Cost models
runningCostModel = crocoddyl.CostModelSum(state, nu=actuationModel.nu)
terminalCostModel = crocoddyl.CostModelSum(state, nu=actuationModel.nu)

# Add a cost for the configuration positions and velocities
xref = np.array([0, 0, 0, 0])  # Desired state
stateResidual = crocoddyl.ResidualModelState(state, xref=xref, nu=actuationModel.nu)
stateCostModel = crocoddyl.CostModelResidual(state, stateResidual)
runningCostModel.addCost("state_cost", cost=stateCostModel, weight=1e-5 / dt)
terminalCostModel.addCost("state_cost", cost=stateCostModel, weight=1000)

# Add a cost on control
controlResidual = crocoddyl.ResidualModelControl(state, nu=actuationModel.nu)
bounds = crocoddyl.ActivationBounds(np.array([-1.0]), np.array([1.0]))
activation = crocoddyl.ActivationModelQuadraticBarrier(bounds)
controlCost = crocoddyl.CostModelResidual(
    state, activation=activation, residual=controlResidual
)
runningCostModel.addCost("control_cost", cost=controlCost, weight=1e-1 / dt)

# Create the action models for the state
runningModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(
        state, actuationModel, runningCostModel
    ),
    dt,
)
terminalModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(
        state, actuationModel, terminalCostModel
    ),
    0.0,
)

Now we define the control problem.

In [ ]:
# Define a shooting problem
q0 = np.zeros((state.nq,))  # Inital joint configurations
q0[0] = np.pi / 2  # Down
v0 = np.zeros((state.nv,))  # Initial joint velocities
x0 = np.concatenate((q0, v0))  # Inital robot state
problem = crocoddyl.ShootingProblem(x0, [runningModel] * T, terminalModel)

Let's test the system with a rollout.

In [ ]:
# Test the problem with a rollout
us = [0.01 * np.ones((1,))] * T
xs = problem.rollout(us)

# Handy to blat up the state and control trajectories
crocoddyl.plotOCSolution(xs, us, show=False, figIndex=99, figTitle="Test rollout")

# Put a grid on the plots
import matplotlib.pyplot as plt

fig = plt.gcf()
axs = fig.axes
for ax in axs:
    ax.grid()

Now we can solve the optimal control problem.

In [ ]:
# Now stabilize the acrobot using FDDP
solver = crocoddyl.SolverFDDP(problem)

# Solve
callbacks = []
callbacks.append(crocoddyl.CallbackLogger())
callbacks.append(crocoddyl.CallbackVerbose())
solver.setCallbacks(callbacks)
solver.solve([], [], 300, False, 1e-5)

We can visualize the trajectory with `meshcat` or using `gepetto-gui` (you will need to install [gepetto-viewer]() and [gepetto-viewer-corba]() and start the process in a separate terminal.)

In [ ]:
# Display using meshcat
robot_display = crocoddyl.MeshcatDisplay(robot, -1, 1, False)
display(robot_display.robot.viewer.jupyter_cell())
robot_display.displayFromSolver(solver)

# Display using gepetto-gui
if False:
    robot_display = crocoddyl.GepettoDisplay(robot, floor=False)
    robot_display.displayFromSolver(solver)

We can plot the trajectory and the solver's convergence properties.

In [ ]:
# Plotting the solution and the DDP convergence
log = solver.getCallbacks()[0]

import matplotlib.pyplot as plt

crocoddyl.plotOCSolution(
    xs=log.xs, us=log.us, show=False, figIndex=1, figTitle="Solution"
)
fig = plt.gcf()
axs = fig.axes
for ax in axs:
    ax.grid(True)

crocoddyl.plotConvergence(
    log.costs,
    log.pregs,
    log.dregs,
    log.grads,
    log.stops,
    log.steps,
    show=False,
    figIndex=2,
)
fig = plt.gcf()
axs = fig.axes
for ax in axs:
    ax.grid(True)

plt.show()